# Change Feed with Azure Functions
## Denormalizing a Non-relational Data Model

In [1]:
%%sql --database acme-webstore --container product
SELECT c.id, c.name, c.price, c.categoryId, c.categoryName, c.tags FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.36 queryCompileTimeInMs : 0.09 queryLogicalPlanBuildTimeInMs : 0.02 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.08 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 6 retrievedDocumentSize : 2214 outputDocumentCount : 6 outputDocumentSize : 996 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,name,price,categoryId,categoryName,tags
0,P1081,V-neck,38,C-TS,T-shirts,"[{'id': 'T-US', 'name': 'ultrasoft'}, {'id': '..."
1,P1082,Crew neck,39,C-TS,T-shirts,"[{'id': 'T-UX', 'name': 'unisex'}]"
2,P1083,Bermuda shorts,42,C-SH,Shorts,"[{'id': 'T-SU', 'name': 'summer'}, {'id': 'T-O..."
3,P1084,Cycling shorts,44,C-SH,Shorts,"[{'id': 'T-OD', 'name': 'outdoor'}, {'id': 'T-..."
4,P1085,Polo collar,27,C-TS,T-shirts,"[{'id': 'T-OD', 'name': 'outdoor'}, {'id': 'T-..."
5,P1086,Boardshorts,32,C-SH,Shorts,"[{'id': 'T-AS', 'name': 'all-season'}]"


In [2]:
%%sql --database acme-webstore --container productMeta
SELECT c.id, c.name, c.type FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.4 queryCompileTimeInMs : 0.09 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.1 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.07 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 7 retrievedDocumentSize : 1798 outputDocumentCount : 7 outputDocumentSize : 369 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,name,type
0,C-TS,T-shirts,category
1,C-SH,Shorts,category
2,T-SU,summer,tag
3,T-US,ultrasoft,tag
4,T-AS,all-season,tag
5,T-OD,outdoor,tag
6,T-UX,unisex,tag


**Start the Denormalization Microservice**

In [3]:
# Replace two items in the productMeta container
database = cosmos_client.get_database_client('acme-webstore')
productMetaContainer = database.get_container_client('productMeta')

# Change category name 'Shorts' to 'Fun shorts'
category = productMetaContainer.read_item('C-SH', 'category')
category['name'] = 'Fun shorts'
productMetaContainer.replace_item('C-SH', category)
print('Modified category name')

# Change tag name from 'outdoor' to 'outdoors'
tag = productMetaContainer.read_item('T-OD', 'tag')
tag['name'] = 'outdoors'
productMetaContainer.replace_item('T-OD', tag)
print('Modified tag name')

Modified category name
Modified tag name


In [4]:
%%sql --database acme-webstore --container productMeta
SELECT c.id, c.name, c.type FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.49 queryCompileTimeInMs : 0.11 queryLogicalPlanBuildTimeInMs : 0.06 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.09 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.06 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 7 retrievedDocumentSize : 1803 outputDocumentCount : 7 outputDocumentSize : 374 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,name,type
0,C-TS,T-shirts,category
1,C-SH,Fun shorts,category
2,T-SU,summer,tag
3,T-US,ultrasoft,tag
4,T-AS,all-season,tag
5,T-OD,outdoors,tag
6,T-UX,unisex,tag


In [5]:
%%sql --database acme-webstore --container product
SELECT c.id, c.name, c.price, c.categoryId, c.categoryName, c.tags FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.61 queryCompileTimeInMs : 0.14 queryLogicalPlanBuildTimeInMs : 0.04 queryPhysicalPlanBuildTimeInMs : 0.04 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.13 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.07 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 6 retrievedDocumentSize : 2229 outputDocumentCount : 6 outputDocumentSize : 1011 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,name,price,categoryId,categoryName,tags
0,P1081,V-neck,38,C-TS,T-shirts,"[{'id': 'T-US', 'name': 'ultrasoft'}, {'id': '..."
1,P1082,Crew neck,39,C-TS,T-shirts,"[{'id': 'T-UX', 'name': 'unisex'}]"
2,P1083,Bermuda shorts,42,C-SH,Fun shorts,"[{'id': 'T-SU', 'name': 'summer'}, {'id': 'T-O..."
3,P1084,Cycling shorts,44,C-SH,Fun shorts,"[{'id': 'T-OD', 'name': 'outdoors'}, {'id': 'T..."
4,P1085,Polo collar,27,C-TS,T-shirts,"[{'id': 'T-OD', 'name': 'outdoors'}, {'id': 'T..."
5,P1086,Boardshorts,32,C-SH,Fun shorts,"[{'id': 'T-AS', 'name': 'all-season'}]"
